# Casa Grande KH-9 Mapping Camera preprocessing


In [3]:
import usgsxplore
import os
import hipp
import geopandas as gpd

## Settings

In [2]:
# path settings
PROJECT_DIRECTORY = "data/casa_grande/kh9_mc_images/1978_03_25"
RAW_IMAGES_DIRECTORY = os.path.join(PROJECT_DIRECTORY, "raw_images")
PREPROCESSED_IMAGES_DIRECTORY = os.path.join(PROJECT_DIRECTORY, "preprocessed_images")

# others settings
QUICKVIEWS_FACTOR = 0.10

In [ ]:
username = os.getenv("USGS_USERNAME") or input("Enter your USGS username: ")
token = os.getenv("USGS_TOKEN") or input("Enter your USGS token: ")

gdf = gpd.read_file(os.path.join(PROJECT_DIRECTORY, "images_footprint.geojson"))
entity_ids = gdf["Entity ID"].to_list()

# connect to the API, download raw images and logout
api = usgsxplore.API(username, token)
api.download("declassii", entity_ids, output_dir=RAW_IMAGES_DIRECTORY)
api.logout()

hipp.tools.optimize_geotifs(RAW_IMAGES_DIRECTORY)
hipp.tools.generate_quickviews(RAW_IMAGES_DIRECTORY, QUICKVIEWS_FACTOR, max_workers=8)

## Preprocessing with ASPy

IMPORTANT : this part is not reproducable cause the pipeline is private. But if you have the code in your machine and you have setup the conda environement you can follow this guide to preprocess the data

```bash
# activate your aspy conda environment
conda activate aspy

cd {PROJECT_DIRECTORY}

# create the preprocessing project
aspy create preproc

# create sym link to access the raw_images without moving them
for img in raw_images/*.tif; do ln -s "$PWD/$img" preproc/data/; done

# create sym link to acces the geojson file to avoid modifying the config
ln -s $PWD/images_footprint.geojson preproc/metadata.gpkg

# create an text file with all ids that will be preprocessed
echo -e "DZB1214-500055L004001\nDZB1214-500055L005001\nDZB1214-500055L006001\nDZB1214-500055L007001" > preproc/ids.txt

# go the preproc directory
cd preproc

# start the preprocessing
aspy preproc
```

In [ ]:
!mkdir -p {PREPROCESSED_IMAGES_DIRECTORY}
!mv {PROJECT_DIRECTORY}/preproc/output/*/*_final.tif {PREPROCESSED_IMAGES_DIRECTORY}

In [ ]:
hipp.tools.generate_quickviews(PREPROCESSED_IMAGES_DIRECTORY, QUICKVIEWS_FACTOR, max_workers=8)
